# Analyze conservatorship case properties

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

### Import cases

In [3]:
cases = pd.read_csv('../input/conservatorship-cases.csv')

In [4]:
cases

,docket,year,number,plaintiff,parcelmbl
0,GD,18,6857,Blight 2 Light Inc.,46-J-341
1,GD,18,6858,Blight 2 Light Inc.,46-N-234
2,GD,18,6858,Blight 2 Light Inc.,46-N-235
3,GD,18,6861,Blight 2 Light Inc,46-J-362
4,GD,18,6864,Blight 2 Light Inc,46-P-19
...,...,...,...,...,...
74,CS,21,79,Brown,45-D-35
75,CS,21,90,East Liberty Development Inc,124-N-8
76,CS,21,96,Perry Hilltop Farms LLC,46-J-334
77,CS,21,96,Perry Hilltop Farms LLC,46-J-336


### Get city, assessment details on case properties

In [5]:
cityproperties = pd.read_sql_query("""SELECT parceleproppgh.parcelid, parcelcentroids.parcelmbl,
parceleproppgh.parceltype, parceleproppgh.parcelstatus,
assessments.class, assessments.usedesc, assessments.conditiondesc
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid
WHERE parcelcentroids.parcelmbl IN %(parcels)s;""", DB, params={'parcels': tuple(cases.parcelmbl)})

In [6]:
cases = pd.merge(cases, cityproperties, on='parcelmbl')

In [7]:
cases[~cases.parcelmbl.isin(cityproperties.parcelmbl)]

,docket,year,number,plaintiff,parcelmbl,parcelid,parceltype,parcelstatus,class,usedesc,conditiondesc


### Status of case properties

In [8]:
status = pd.pivot_table(cases, index=['parceltype', 'parcelstatus'], values='parcelid', aggfunc=len)
status = status.rename(columns={ 'parcelid': 'count' }).reset_index()
status['pct'] = status['count'] / status['count'].sum()

status

,parceltype,parcelstatus,count,pct
0,CDC Property Reserve,Sale Pending,5,0.064103
1,Hold For Study,Hold For Study,8,0.102564
2,PLB Transfer,Transferred,1,0.012821
3,Public Sale,Available for Sale,13,0.166667
4,Public Sale,Sale Pending,4,0.051282
5,URA Transfer,Available for Sale,43,0.551282
6,URA Transfer,Sale Pending,2,0.025641
7,Unknown Public Use,Hold For Study,2,0.025641


In [9]:
holdstatus = status[status.parceltype.isin(['Hold For Study', 'Unknown Public Use', 'URA Transfer'])].copy()
holdstatus.loc['Total'] = holdstatus.sum(numeric_only=True)

holdstatus

,parceltype,parcelstatus,count,pct
1,Hold For Study,Hold For Study,8.0,0.102564
5,URA Transfer,Available for Sale,43.0,0.551282
6,URA Transfer,Sale Pending,2.0,0.025641
7,Unknown Public Use,Hold For Study,2.0,0.025641
Total,NaN,NaN,55.0,0.705128


### Land use of case properties

In [10]:
pd.pivot_table(cases, index=['class', 'usedesc', 'conditiondesc'], values='parcelid', aggfunc=len)

parcelid
class usedesc                     conditiondesc          
G     FEDERAL GOVERNMENT                                1
      MUNICIPAL GOVERNMENT                             16
R     RES AUX BUILDING (NO HOUSE)                       1
      ROWHOUSE                    AVERAGE               1
                                  FAIR                  3
                                  POOR                  3
      SINGLE FAMILY               AVERAGE               1
                                  FAIR                 14
                                  POOR                  5
                                  VERY POOR             4
      TOWNHOUSE                   AVERAGE               1
      TWO FAMILY                  FAIR                  1
                                  UNSOUND               1
      VACANT LAND                                      26